### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?




#### Solution 

1.
The given problem can be formulated as a Least Squares problem as follows:

$$
\begin{aligned}
 \underset{A_{12},A_{21}}{\text{minimize: }} \sum_{i=1}^{N=11}(p(x_i;A_{12},A_{21})-p_i)^2
\end{aligned}
$$

<br>
2.

Saturation pressure for water:

$$
\log_{10}(p_{water}^{sat}) = a_1 - \frac{a_2}{T + a_3} = 8.07131 - \frac{1730.63}{20 + 233.426} = 1.24237
$$

<br>
$$
\Rightarrow p_{water}^{sat} = 17.47325
$$


Saturation pressure for 1,4dioxane:

$$
\log_{10}(p_{1,4dioxane}^{sat}) = a_1 - \frac{a_2}{T + a_3} = 7.43155 - \frac{1554.679}{20 + 240.337} = 1.45975
$$

<br> 	
$$
\Rightarrow p_{water}^{sat} = 28.82409
$$

<br>
With $x_2=1-x_1$, the equilibrium relation becomes:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}(1-x_1)}{A_{12}x_1+A_{21}(1-x_1)}\right)^2\right)p_{water}^{sat}\\
& + (1-x_1)\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}(1-x_1)}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$


In [65]:
import numpy as np
import torch 



def saturation_pressure(T, a1, a2, a3):
    """Compute the saturation pressure."""

    log_p = a1 - (a2/(T + a3))
    p = 10**log_p
    return p


def gradient_descent(x1, p, a, eps, max_iter):
    """ """

    # Randomly initialize weights
    # A12 = torch.randn((), requires_grad=True, device=device, dtype=torch.float) 
    # A21 = torch.randn((), requires_grad=True, device=device, dtype=torch.float) 

    A = torch.randn(2, requires_grad=True, device=device, dtype=torch.float)

    error = float('inf')
    iter = 0

    while error > eps and iter < max_iter:

        # Compute p prediction (Forward pass)
        p_pred = x1*torch.exp(A[0]*(A[1]*(1-x1)/(A[0]*x1+A[1]*(1-x1)))**2)*water_sat_p + \
                (1-x1)*torch.exp(A[1]*(A[0]*x1/(A[0]*x1+A[1]*(1-x1)))**2)*dioxane_sat_p
        
        # Compute loss
        loss = (p_pred - p).pow(2).sum()

        # Print loss every 100 iterations
        if iter % 100 == 0:
            print('Iteration: {}, Loss: {} '.format(iter, loss.item()))
        
        # Compute gradient of the loss wrt all the learnable parameters of the model
        loss.backward()

        # Current error
        error = torch.linalg.norm(A)          


        # Update the weights using Gradient Descent
        with torch.no_grad():  # We don't need the GD algorithm itself to be differentiable wrt A
            A -= a * A.grad
        
            # Clear the gradients (so they don't accumulate)
            A.grad.zero_()
        
        
        iter += 1
    
    print('\nNumber of iterations: {}'.format(iter))
    print('Final loss value: {}'.format(loss.data.numpy()))
    print('Final error: {}'.format(error))

    return A.data





if __name__ == '__main__':

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Parameters
    # --------------------------------------------------------------------------
    eps = 1e-3                       # Tolerance (termination criterion)
    max_iter = 20000                 # Maximum iterations before halting

    # Compute saturation pressures
    # --------------------------------------------------------------------------
    water_sat_p = saturation_pressure(T=20, a1=8.07131, a2=1730.63, a3=233.426)
    dioxane_sat_p = saturation_pressure(T=20, a1=7.43155, a2=1554.679, a3=240.337)


    
    # Input data
    x1 = torch.tensor([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], device=device, dtype=torch.float)
    # Output data
    p = torch.tensor([28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5], device=device, dtype=torch.float)

    sol = gradient_descent(x1, p, 0.001, eps, max_iter)


    print('The solution is: A = {}'.format(sol.data.numpy()))

    




Iteration: 0, Loss: 1536.2203369140625 
Iteration: 100, Loss: 0.6701919436454773 
Iteration: 200, Loss: 0.6701900362968445 
Iteration: 300, Loss: 0.6701900362968445 
Iteration: 400, Loss: 0.6701900362968445 
Iteration: 500, Loss: 0.6701900362968445 
Iteration: 600, Loss: 0.6701900362968445 
Iteration: 700, Loss: 0.6701900362968445 
Iteration: 800, Loss: 0.6701900362968445 
Iteration: 900, Loss: 0.6701900362968445 
Iteration: 1000, Loss: 0.6701900362968445 
Iteration: 1100, Loss: 0.6701900362968445 
Iteration: 1200, Loss: 0.6701900362968445 
Iteration: 1300, Loss: 0.6701900362968445 
Iteration: 1400, Loss: 0.6701900362968445 
Iteration: 1500, Loss: 0.6701900362968445 
Iteration: 1600, Loss: 0.6701900362968445 
Iteration: 1700, Loss: 0.6701900362968445 
Iteration: 1800, Loss: 0.6701900362968445 
Iteration: 1900, Loss: 0.6701900362968445 
Iteration: 2000, Loss: 0.6701900362968445 
Iteration: 2100, Loss: 0.6701900362968445 
Iteration: 2200, Loss: 0.6701900362968445 
Iteration: 2300, Loss: 

---
### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).



#### Solution 

